## **Advanced Classification using Machine Learning in HealthCare**

## Introduction

X-rays are widely used in medical practice. They can be used to identify various diseases. However, a diagnosis depends on a doctor's experience, which can lead to improper treatment. Modern methods of artificial intelligence and pattern recognition make it possible to create expert systems that allow you to establish a diagnosis automatically.

This lab will show you how to upload images, transform them, and determine the basic features that underlie diseases classification.

Two different approaches to the classification of images (diseases) will be shown:
1. Different classical methods and their comparison 
2. Convolutional Neural Networks.

## Materials and methods

In this lab, we will learn the basic methods of images classifications. The laboratory consists of four stages:
* Download and preliminary transformation of images
* Creating image features
* Comparing different classical classification methods
* Building and fitting Convolutional Neural Network

The statistical data was obtained from https://www.kaggle.com/pranavraikokte/covid19-image-dataset under [CC BY-SA 4.0](https://creativecommons.org/licenses/by-sa/4.0/) license.

## Prerequisites

* [Python](https://www.python.org)
* [os](https://docs.python.org/3/library/os.html)
* [numpy](https://numpy.org)
* [glob](https://docs.python.org/3/library/glob.html)
* [SeaBorn](https://seaborn.pydata.org)
* [Matplotlib](https://matplotlib.org)
* [mahotas](https://mahotas.readthedocs.io/en/latest/)
* [keras](https://keras.io)
* [scikit-learn](https://scikit-learn.org)
* [pandas](https://pandas.pydata.org)

## Objectives

* Download and transform images.
* Create features of images.
* Build different classification models.
* Build CNN models.
* Build a diagnosis based on X-ray photos.

# Download and preliminary transformation of images

## Required libraries installation

We need to install additional libraries and upgrade existing ones in the lab.

In [ ]:
pip install mahotas

In [ ]:
conda install scikit-learn

In [ ]:
conda install -c conda-forge tensorflow --yes

In [ ]:
conda install -c conda-forge keras --yes

In [ ]:
conda config --add channels defaults

## Required libraries import

Here we are going to use Mahotas for image processing and Keras library for creating our CNN model and its training. We will also use Matplotlib and Seaborn for visualizing our dataset to gain a better understanding of the images we are going to handle.
We will also use libraries os and glob to work with files and folders. NumPy will be applied for arrays of images. Scikit-Learn will be used for classical classification models. And we will take Pandas for present comparison of classifiers.

In [ ]:
import mahotas as mh
import seaborn as sns
from matplotlib import pyplot as plt 
from glob import glob
import os
import numpy as np
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler 

#Classifiers
from sklearn.linear_model import LogisticRegression 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import ConfusionMatrixDisplay

## Data loading

In [ ]:
import skillsnetwork

await skillsnetwork.prepare("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/data-science-in-health-care-advanced-machine-learning-classification/LargeData/Covid19-dataset.zip", overwrite=True)
await skillsnetwork.prepare("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/data-science-in-health-care-advanced-machine-learning-classification/NN.zip", overwrite=True)

Images have to be of the same size for classification. In order to achieve this, let's create a global variable that will determine the size (height and width) for image resizing. Both are 224 in our case.

In [ ]:
IMM_SIZE = 224

For convenience, we create a function that downloads and displays all the pictures from a specified directory.
In order to classify images, all pictures should be placed in subdirectories. The names of these subdirectories are actually class names.
In our case, the images are X-rays which should be placed into subfolders with the names of diagnoses.
For example, a COVID subfolder is to contain X-rays of people with this disease.
First of all, we should create a list of subfolders, which is a list of possible disease classes. In our case, it will be: Normal, COVID, Viral Pneumonia.

Next, we need to create a list of all images.
After that, let's download and preprocess all the images:
1. Download by [mahotas.imread()](https://mahotas.readthedocs.io/en/latest/io.html)
2. It is necessary to resize the images to (IMM_SIZE x IMM_SIZE). If an image is gray-colored it is presented as a 2D matrix: (high, width). jpg and png images are 3D (high, width, 3 or 4). To do this, we should use: [mahotas.resize_to()](https://github.com/luispedro/mahotas/blob/master/mahotas/resize.py)
3. If the third parameter of an image shape equals 4, it means alpha channel. We can remove it using slice image[:,:,:3].
4. Then we need to transform all the images to gray 2D format by: [mahotas.colors.rgb2grey()](https://mahotas.readthedocs.io/en/latest/color.html)

The function returns an array of tuples [image, class name].


In [ ]:
def get_data(folder):
    
    class_names = [f for f in os.listdir(folder) if not f.startswith('.')] # create a list of SubFolders
    images_data = []
    labels = []
    class_names = ['Covid', 'Normal', 'Viral Pneumonia']
    print(class_names)
    for t, f in enumerate(class_names):
        images = glob(folder + "/" + f + "/*") # create a list of files
        print("Downloading: ", f)
        fig = plt.figure(figsize = (50,50)) 
        for im_n, im in enumerate(images):
            plt.gray() # set grey colormap of images
            image = mh.imread(im)
            if len(image.shape) > 2:
                image = mh.resize_to(image, [IMM_SIZE, IMM_SIZE, image.shape[2]]) # resize of RGB and png images
            else:
                image = mh.resize_to(image, [IMM_SIZE, IMM_SIZE]) # resize of grey images    
            if len(image.shape) > 2:
                image = mh.colors.rgb2grey(image[:,:,:3], dtype = np.uint8)  # change of colormap of images alpha chanel delete
            plt.subplot(int(len(images)/5)+1,5,im_n+1) # create a table of images
            plt.imshow(image)
            images_data.append(image)
            labels.append(f)
        plt.show()

    # Convert images to numpy array after ensuring all have same shape
    return np.array(images_data), np.array(labels)

For training and testing, all the pictures should be divided into training and test groups and located in separate folders. Let's upload all the images to the corresponding arrays.


In [ ]:
d = "Covid19-dataset/train"
train_images, train_labels = get_data(d)

d = "Covid19-dataset/test"
val_images, val_labels = get_data(d)

In [ ]:
print("Train images shape:", train_images.shape) # Size of the training DataSet
print("Train labels shape:", train_labels.shape) # Size of the training labels
print("Test images shape:", val_images.shape) # Size of the test DataSet
print("Test labels shape:", val_labels.shape) # Size of the test labels
print("Image size:", train_images[0].shape) # Size of a single image

As you can see, the training DataSet consists of 251 images and the test one consists of 66 images. All the images are in grey 2D (224x224) format.


## Data visualization 

Let’s visualize our data and see what exactly we are working with. We use Seaborn to plot the number of images in all the classes. You can see what the output looks like.


In [ ]:
l = train_labels
sns.set_style('darkgrid')
sns.countplot(l)
plt.title('Distribution of Classes in Training Data')
plt.xlabel('Class')
plt.ylabel('Count')

Let us also visualize the first image from the Viral Pneumonia and Covid classes in the training DataSet:


In [ ]:
viral_pneumonia_indices = np.where(train_labels == 'Viral Pneumonia')[0]


plt.figure(figsize = (5,5))
plt.imshow(train_images[viral_pneumonia_indices[0]])
plt.title('Viral Pneumonia')

In [ ]:
covid_indices = np.where(train_labels == 'Covid')[0]

plt.figure(figsize = (5,5))
plt.imshow(train_images[covid_indices[0]])
plt.title('Covid')

## Image features creation

To classify objects, you need to transform the data set so that the input is a set of features and the output is an object class. An image is a matrix of pixels. Each pixel is a color. Therefore, it is impossible to submit a bare picture into a classical classifier's input. It is necessary to turn each picture into a set of certain features.

Mahotas makes it easy to calculate features of an image.
The corresponding functions are found in the [mahotas.features](https://mahotas.readthedocs.io/en/latest/) submodule. The Haralick set of texture features is well known. Like many image processing algorithms, it is named for its inventor. The features are based on textures, that is, they distinguish between structured and unstructured images, as well as various repetitive structures. With the help of Mahotas, these features are calculated very easily:
haralick_features = mh.features.haralick (image)
haralick_features_mean = np.mean (haralick_features, axis = O)
haralick_features_all = np.ravel (haralick_features)
The mh.features.haralick function returns a 4 x 13 array wiсh should be transformad into 1D by [NumPy.ravel()](https://numpy.org/doc/stable/reference/generated/numpy.ravel.html). The first dimension is the four possible directions in which the features are calculated (vertical, horizontal, and two diagonals). If we are not interested in any particular direction, then we can average the features in all directions (in the code above, this variable is called haralick_features_mean). Alternatively, you can use all the characteristics individually (variable haralick_features_all). The choice depends on the properties of a particular dataset. We decided that the vertical and horizontal features should be stored separately in our case, so we use haralick_features_all.

We should make a function for the DataSet features creation.


In [ ]:
def create_features(images, labels):
    features = []
    for image in images:
        features.append(mh.features.haralick(image).ravel())
    features = np.array(features)
    return features, labels

In [ ]:
features_train, labels_train = create_features(train_images, train_labels)
features_test, labels_test = create_features(val_images, val_labels)

# Comparing different classical classification methods

If we want to compare some classificators, we should use a Pipeline.
A pipeline helps to chain multiple estimators into a single one. This is useful as there is often a fixed number of steps in data processing, for example, feature selection, normalization and classification. A pipeline serves multiple purposes here:

You only have to call fit() once to evaluate a whole sequence of estimators.

You can grid search over parameters of all estimators in the pipeline at once.

Pipelines help to avoid leaking statistics from your test data into the trained model in cross-validation by ensuring that the same samples are used to train the transformers and predictors.
All estimators in a pipeline, except the last one, should be transformers (i.e. should have a transform method). The last estimator may be of any type (transformer, classifier, etc.).

The [sklearn.pipeline](https://scikit-learn.org/stable/modules/classes.html?highlight=pipeline#module-sklearn.pipeline) module implements utilities to build a composite estimator, as a chain of transforms and estimators.

In order to test how it works, we will use LogisticRegression.


In [ ]:
clf = Pipeline([('preproc', StandardScaler()), ('classifier', LogisticRegression())])
clf.fit(features_train, labels_train)
scores_train = clf.score(features_train, labels_train)
scores_test = clf.score(features_test, labels_test)
print('Training DataSet accuracy: {: .1%}'.format(scores_train), 'Test DataSet accuracy: {: .1%}'.format(scores_test))
ConfusionMatrixDisplay.from_estimator(clf, features_test, labels_test)  
plt.show()

As you can see, the results are not bad.
Conflusion matrix shows us how many mistaken predictions we got.
It allows us to check other classifiers and compare the results.
We will test:
* [Logistic Regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html?highlight=logistic%20regression#sklearn.linear_model.LogisticRegression)
* [Nearest Neighbors](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html?highlight=nearest%20neighbors#sklearn.neighbors.NearestNeighbors)
* [Linear SVM](https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVR.html?highlight=linear%20svm#sklearn.svm.LinearSVR)
* [RBF SVM](https://scikit-learn.org/stable/modules/generated/sklearn.gaussian_process.kernels.RBF.html?highlight=rbf#sklearn.gaussian_process.kernels.RBF)
* [Gaussian Process](https://scikit-learn.org/stable/modules/generated/sklearn.gaussian_process.GaussianProcessClassifier.html?highlight=gaussianprocessclassifier#sklearn.gaussian_process.GaussianProcessClassifier)
* [Decision Tree](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html?highlight=decisiontreeclassifier#sklearn.tree.DecisionTreeClassifier)
* [Random Forest](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html?highlight=randomforestclassifier#sklearn.ensemble.RandomForestClassifier)
* [Multi-layer Perceptron classifier](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html?highlight=mlpclassifier#sklearn.neural_network.MLPClassifier)
* [Ada Boost](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html?highlight=adaboostclassifier#sklearn.ensemble.AdaBoostClassifier)
* [Naive Bayes](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html?highlight=gaussiannb#sklearn.naive_bayes.GaussianNB)
* [Quadratic Discriminant Analysis](https://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.QuadraticDiscriminantAnalysis.html?highlight=quadraticdiscriminantanalysis#sklearn.discriminant_analysis.QuadraticDiscriminantAnalysis)


In [ ]:
# Create a DataFrame to display the results
results = pd.DataFrame({
    'Classifier': names,
    'Training Accuracy': scores_train,
    'Testing Accuracy': scores_test
})

# Display the results as a table
print("\nClassifier Accuracy Comparison:")
print(results.sort_values(by='Testing Accuracy', ascending=False))

# Plot the results
plt.figure(figsize=(12, 6))
results_sorted = results.sort_values(by='Testing Accuracy', ascending=False)
x = np.arange(len(names))
width = 0.35

plt.bar(x - width/2, results_sorted['Training Accuracy'], width, label='Training Accuracy')
plt.bar(x + width/2, results_sorted['Testing Accuracy'], width, label='Testing Accuracy')

plt.xlabel('Classifier')
plt.ylabel('Accuracy')
plt.title('Classifier Performance Comparison')
plt.xticks(x, results_sorted['Classifier'], rotation=45, ha='right')
plt.legend()
plt.tight_layout()
plt.show()

# Find the best classifier
best_idx = np.argmax(scores_test)
print(f"\nBest classifier: {names[best_idx]} with test accuracy: {scores_test[best_idx]:.2%}")

# Let's see the confusion matrix for the best classifier
best_clf = Pipeline([('preproc', StandardScaler()), ('classifier', classifiers[best_idx])])
best_clf.fit(features_train, labels_train)
print("\nConfusion Matrix for the Best Classifier:")
ConfusionMatrixDisplay.from_estimator(best_clf, features_test, labels_test)
plt.show()

Let's print the results as a table.


In [ ]:
res = pd.DataFrame(index=names)
res['Train'] = scores_train  # Use correct column name from the start
res['Test'] = scores_test    # Use correct column name from the start
res.index.name = "Classifier accuracy"
pd.options.display.float_format = '{:.2f}'.format  # Format to 2 decimal places
print(res)

You can see that the calculations are very fast and that Logistic Regression and Neural Network show the best result for the test DataSet.

Let's compare the results on a plot.

In [ ]:
# Plot the comparison of training and testing accuracies
x = np.arange(len(names))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots(figsize=(12, 6))  # Create a larger figure for better readability
rects1 = ax.bar(x - width/2, scores_train, width, label='Train')
rects2 = ax.bar(x + width/2, scores_test, width, label='Test')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Accuracy')
ax.set_title('Accuracy of Classifiers')
ax.set_xticks(x)
ax.set_xticklabels(names, rotation=45, ha='right')
ax.legend()

# Add a grid for easier comparison
ax.grid(True, axis='y', linestyle='--', alpha=0.7)

# Add value annotations on top of bars
def autolabel(rects):
    for rect in rects:
        height = rect.get_height()
        ax.annotate(f'{height:.2f}',
                   xy=(rect.get_x() + rect.get_width()/2, height),
                   xytext=(0, 3),  # 3 points vertical offset
                   textcoords="offset points",
                   ha='center', va='bottom', fontsize=8)

autolabel(rects1)
autolabel(rects2)

fig.tight_layout()
plt.show()

# Building and fitting Convolutional Neural Network


## Required libraries import

We will use Keras library for creating and training our model.


In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix

## Data preprocessing and data augmentation

What is different about [Convolutional Neural Networks](https://en.wikipedia.org/wiki/Convolutional_neural_network) is that we can submit images directly to the input. However, these images also require pre-processing. 

In particular, it is necessary to normalize the pixels color, i.e. to normalize them from the range [0, 255) to [0, 1).

You also need to change the dimension of the input images because of Keras framework. 

Image classes should be of numeric type instead of string.

The code below makes the necessary transformations.


In [ ]:
# Initialize empty lists for training and validation data
x_train = []
y_train = []
x_val = []
y_val = []

for feature, label in zip(train_images, train_labels):
    x_train.append(feature)
    y_train.append(label)

for feature, label in zip(val_images, val_labels):
    x_val.append(feature)
    y_val.append(label)

# Normalize the data (pixel values from [0,255] to [0,1])
x_train = np.array(x_train) / 255
x_val = np.array(x_val) / 255

# Reshape images to add channel dimension for CNN input
# (samples, height, width, channels)
x_train = x_train.reshape(-1, IMM_SIZE, IMM_SIZE, 1)
x_val = x_val.reshape(-1, IMM_SIZE, IMM_SIZE, 1)

# Create a mapping from string labels to numeric indices
lab = {}
for i, l in enumerate(sorted(set(y_train))):
    lab[l] = i

# Convert string labels to numeric indices
y_train = np.array([lab[l] for l in y_train])
y_val = np.array([lab[l] for l in y_val])

In [ ]:
print("Shape of the input DataSet:", x_train.shape)
print("Shape of the output DataSet:", y_train.shape)
print("Dictionary of classes:", lab)

## Data augmentation on the training data

We should perform data augmentation to fit our model better.


In [ ]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=30,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range=0.2,  # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally
        height_shift_range=0.1,  # randomly shift images vertically
        horizontal_flip=True,  # randomly flip images horizontally
        vertical_flip=False,  # don't flip images vertically
        brightness_range=[0.8, 1.2],  # randomly adjust brightness
        fill_mode='nearest')  # strategy for filling points outside boundaries

datagen.fit(x_train)

## Model defining

Let’s define a simple CNN model with 3 Convolutional layers followed by max-pooling layers. A dropout layer is added after the 3rd maxpool operation to avoid overfitting.


In [ ]:
from keras.layers import Input

# Define model using the recommended approach
model = Sequential([
    # Input layer explicitly defined
    Input(shape=(IMM_SIZE, IMM_SIZE, 1)),
    
    # First convolutional block
    Conv2D(32, 1, padding="same", activation="relu"),
    MaxPool2D(),
    
    # Second convolutional block
    Conv2D(32, 1, padding="same", activation="relu"),
    MaxPool2D(),
    
    # Third convolutional block
    Conv2D(64, 1, padding="same", activation="relu"),
    MaxPool2D(),
    Dropout(0.4),
    
    # Fully connected layers
    Flatten(),
    Dense(128, activation="relu"),
    Dense(3, activation="softmax")
])

# Display the model summary
model.summary()

Let’s compile the model now using Adam as our optimizer and SparseCategoricalCrossentropy as the loss function. We are using a lower learning rate of 0.000001 for a smoother curve.


In [ ]:
opt = Adam(learning_rate=0.000001)

# Compile the model with appropriate loss function
model.compile(
    optimizer=opt,
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

Now, let’s train our model for **2000** epochs.
Admittedly, the fitting process is very slow. Therefore, we saved the fitted model to a file.
To save time, we will upload the fitted model.
If you would like, you can change the parameter **fitting to True** in order to refit the model.


In [ ]:
# Set training parameters
fitting = False  # Change to True if you want to train the model
fitting_save = True  # Whether to save the trained model (when fitting=True)
epochs = 2000  # Number of epochs to train for

# Import necessary libraries for model loading/saving
import pickle
import os
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

# Ensure tf version is printed to help with debugging
print(f"TensorFlow version: {tf.__version__}")

if fitting:
    print(f"Training model for {epochs} epochs...")
    
    # Add callbacks for better training
    callbacks = [
        # Save best model based on validation accuracy
        ModelCheckpoint(
            "best_model.h5", 
            monitor="val_accuracy",
            save_best_only=True,
            mode="max"
        ),
        # Early stopping to prevent overfitting
        EarlyStopping(
            monitor="val_accuracy",
            patience=50,  # Generous patience for long training
            restore_best_weights=True
        ),
        # Reduce learning rate when training plateaus
        ReduceLROnPlateau(
            monitor="val_loss",
            factor=0.5,
            patience=20,
            min_lr=1e-7
        )
    ]
    
    # Train the model with data augmentation
    history = model.fit(
        datagen.flow(x_train, y_train, batch_size=32),
        epochs=epochs,
        validation_data=(x_val, y_val),
        callbacks=callbacks,
        verbose=1
    )
    
    if fitting_save:
        print("Saving model...")
        
        # First, the modern recommended way - save complete model
        model.save("covid_model_complete")
        
        # Also save as weights + architecture for compatibility
        model.save_weights("covid_model_weights.h5")
        
        # Save model architecture as JSON
        with open("covid_model_architecture.json", "w") as f:
            f.write(model.to_json())
            
        # Save training history and class mapping
        with open('training_history.pickle', 'wb') as f:
            pickle.dump(history.history, f)
        with open('class_mapping.pickle', 'wb') as f:
            pickle.dump(lab, f)
            
        print("Model saved successfully")
        
else:
    print("Using pretrained model...")
    
    # If we're not training, we'll always rebuild the model
    # This ensures compatibility and avoids version issues
    
    # First, check if we have history files to load
    if os.path.exists('history.pickle'):
        with open('history.pickle', 'rb') as f:
            history = pickle.load(f)
        print("Loaded training history")
    else:
        print("No training history found")
    
    # Recreate the model architecture - this is our fallback approach
    print("Creating model with pretrained architecture...")
    
    # Model is already defined earlier in the notebook
    # Compile with slightly higher learning rate since we're not training from scratch
    opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(
        optimizer=opt,
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
        metrics=['accuracy']
    )
    
    # Try to load weights if they exist
    try:
        if os.path.exists("model.h5"):
            model.load_weights("model.h5")
            print("✅ Successfully loaded pretrained weights")
        else:
            print("⚠️ Could not find pretrained weights")
            print("You may want to set fitting=True to train the model from scratch")
    except Exception as e:
        print(f"⚠️ Error loading weights: {e}")
        print("You may need to set fitting=True to train the model from scratch")

# Print quick summary of model status
print("\nModel summary:")
model.summary()

## Results evaluation

We will plot our training and validation accuracy along with the training and validation loss.


In [ ]:
acc = history['accuracy']
val_acc = history['val_accuracy']
loss = history['loss']
val_loss = history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(15, 15))
plt.subplot(2, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

Let’s see what the curve looks like.
You can see that the accuracy of the training and validation sets is the same. The loss function of validation and training sets is stable. It means that our CNN is fitted well and can be used for classification.

We can print out the classification report to see the precision and accuracy using [model.predict_classes()]() and [classification_report()]().

Also we can create a confusion matrix. Unfortunately, Keras framework does not have plot_confusion_matrix() function. Therefore, we have to create it using Pandas and [Seaborn.heatmap()]()


In [ ]:
# Get model predictions
predictions_prob = model.predict(x_val)  # This returns class probabilities
predictions = np.argmax(predictions_prob, axis=1)  # Get the class with highest probability

# Print classification report
print("Classification Report:")
print(classification_report(y_val, predictions, target_names=list(lab.keys())))

# Create and display confusion matrix as a table
cm = pd.DataFrame(confusion_matrix(y_val, predictions))
# Convert numeric indices back to class names
class_mapping = {v: k for k, v in lab.items()}  # Reverse mapping from index to class name
cm.index = [f"Predicted {class_mapping[i]}" for i in range(len(lab))]
cm.columns = [f"True {class_mapping[i]}" for i in range(len(lab))]
print("\nConfusion Matrix:")
print(cm)

# Create a heatmap visualization of the confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(confusion_matrix(y_val, predictions), annot=True, fmt='d',
            xticklabels=list(lab.keys()), yticklabels=list(lab.keys()),
            cmap='Blues', linewidths=1, linecolor='black')
plt.xlabel("True labels")
plt.ylabel("Predicted labels")
plt.title("Confusion Matrix")
plt.tight_layout()
plt.show()

# Calculate and display overall accuracy
accuracy = np.mean(predictions == y_val)
print(f"\nOverall Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")

# Display examples of misclassified images
misclassified = np.where(predictions != y_val)[0]
if len(misclassified) > 0:
    print(f"\nShowing {min(4, len(misclassified))} misclassified examples:")
    plt.figure(figsize=(15, 4))
    for i, idx in enumerate(misclassified[:4]):
        plt.subplot(1, 4, i+1)
        plt.imshow(x_val[idx].reshape(IMM_SIZE, IMM_SIZE), cmap='gray')
        plt.title(f"True: {class_mapping[y_val[idx]]}\nPred: {class_mapping[predictions[idx]]}")
        plt.axis('off')
    plt.tight_layout()
    plt.show()
else:
    print("\nNo misclassified examples found!")

In [ ]:
predictions_train = model.predict(x_train)  # Get probabilities
predictions_train_classes = np.argmax(predictions_train, axis=1)  # Convert to class indices
scores_train = np.mean(predictions_train_classes == y_train)  # Calculate accuracy

# For validation/test set
predictions_val = model.predict(x_val)  # Get probabilities
predictions_val_classes = np.argmax(predictions_val, axis=1)  # Convert to class indices
scores_test = np.mean(predictions_val_classes == y_val)  # Calculate accuracy

print('Training DataSet accuracy: {: .1%}'.format(scores_train), 'Test DataSet accuracy: {: .1%}'.format(scores_test))

As you can see, the CNN shows better results than classical models. However, fitting takes much longer.

And now, on your own, try to create a function that will establish a diagnosis based on the CNN. 


In [ ]:
def diagnosis(file):
    """
    Diagnose an X-ray image using the trained CNN model.
    
    Parameters:
    file (str): Path to the image file
    
    Returns:
    str: The diagnosis (Covid, Normal, or Viral Pneumonia)
    """
    # Download image
    image = mh.imread(file)
    
    # Prepare image to classification
    if len(image.shape) > 2:
        image = mh.resize_to(image, [IMM_SIZE, IMM_SIZE, image.shape[2]])
        # Remove alpha channel if present
        image = mh.colors.rgb2grey(image[:,:,:3], dtype=np.uint8)
    else:
        image = mh.resize_to(image, [IMM_SIZE, IMM_SIZE])
        
    # Show image
    plt.figure(figsize=(5, 5))
    plt.imshow(image, cmap='gray')
    plt.title('Input X-ray image')
    plt.axis('off')
    plt.show()

    # Load model
    # Note: We assume the model is already loaded as 'model' 
    # and the class mapping dictionary 'lab' is available from previous cells
    
    # Create reverse mapping from index to class name
    class_mapping = {v: k for k, v in lab.items()}
    
    # Normalize the data
    image_normalized = image / 255.0
    
    # Reshape input images
    image_reshaped = image_normalized.reshape(1, IMM_SIZE, IMM_SIZE, 1)
    
    # Predict the diagnosis
    prediction = model.predict(image_reshaped)
    prediction_class = np.argmax(prediction, axis=1)[0]
    
    # Get prediction probabilities for each class
    probabilities = prediction[0]
    
    # Find the name of the diagnosis
    diag = class_mapping[prediction_class]
    
    # Print detailed results
    print(f"Diagnosis: {diag}")
    print("\nProbabilities:")
    for i, (class_name, prob) in enumerate(zip(lab.keys(), probabilities)):
        print(f"{class_name}: {prob:.2%}")
        
    # Visualize confidence levels
    plt.figure(figsize=(10, 5))
    plt.bar(lab.keys(), probabilities, color='skyblue')
    plt.xlabel('Diagnosis')
    plt.ylabel('Confidence')
    plt.title('Prediction Confidence')
    plt.ylim(0, 1)
    plt.axhline(y=0.5, color='r', linestyle='--', alpha=0.5)
    plt.grid(axis='y', alpha=0.3)
    for i, v in enumerate(probabilities):
        plt.text(i, v + 0.02, f"{v:.2%}", ha='center')
    plt.tight_layout()
    plt.show()
    
    return diag

In [ ]:
print ("Diagnosis is:", diagnosis("Covid19-dataset/test/Covid/0120.jpg"))
print ("Diagnosis is:", diagnosis("Covid19-dataset/test/Normal/0105.jpeg"))
print ("Diagnosis is:", diagnosis("Covid19-dataset/test/Viral Pneumonia/0111.jpeg"))

## Conclusions

This lab has revealed for us how to create an expert system that allows us to obtain diagnosis based on X-Rays images, using different classificators. This principles can be used for any type of X-Rays, not only for COVID diagnostics.

During this laboratory work, we have explored images downloading and transforming. We have learned how to extract features of images and build/fit/test/compare sets of classificators. Also we got to know how to create and fit Convolutional Neural Networks. We have compared accuracy of different classificators. 
